# Selección del Mejor Modelo para el Proyecto F5 Airline (Tecnicas de Validación Cruzada y Ensemble Learning)

Este código implementa un proceso completo de carga de datos, preprocesamiento, evaluación de modelos de machine learning, selección del mejor modelo, entrenamiento final con todos los datos, almacenamiento del modelo en disco, y finalmente inserción de las métricas y parámetros del modelo en una base de datos SQLite.

También trabaja con Pipelines, y técnicas de Validación Cruzada y Ensemble Learning (StackingClassifier) 


## Importación de Librerías

In [20]:
# Librerías principales para manipulación de datos
import pandas as pd  # Manejo de datos en formato tabular (DataFrames)
import numpy as np  # Funciones matemáticas y operaciones con arrays
import sqlite3  # Para conectarse a la base de datos SQLite
import os  # Para manejar el sistema de archivos
import joblib  # Para guardar y cargar modelos entrenados
from datetime import datetime, timedelta  # Para obtener la fecha y hora actual 
import time  # Para medir el tiempo de procesamiento

# Librerías para construir pipelines y preprocesamiento
from sklearn.pipeline import Pipeline  # Para crear pipelines que incluyan pasos de preprocesamiento y modelo
from sklearn.preprocessing import StandardScaler  # Para escalar variables numéricas
from sklearn.compose import ColumnTransformer  # Para aplicar diferentes transformaciones a diferentes tipos de columnas

# Modelos de machine learning
from sklearn.ensemble import RandomForestClassifier, StackingClassifier  # Algoritmo de clasificación basado en árboles y Stacking (Técnica de Ensemble Learning)
from catboost import CatBoostClassifier  # Algoritmo CatBoost, un modelo de boosting que maneja variables categóricas internamente
from xgboost import XGBClassifier  # Algoritmo XGBoost, un modelo de boosting eficiente que tambien sirve para tratar variables categoricas

# Librerías para validación cruzada y evaluación de modelos
from sklearn.model_selection import cross_validate, StratifiedKFold  # Validación cruzada y división estratificada de los datos
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score  # Métricas para evaluar el rendimiento de los modelos

# Configuración cuaderno Jupyter: Mostrar todas las columnas y ajustar ancho de las columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

## Función para Conectar con la Base de Datos
- Objetivo: Establecer una conexión con la base de datos SQLite.
- Razón: Los resultados de los modelos y sus parámetros serán almacenados en la base de datos para futuras consultas.

In [21]:
# Función para conectar a la base de datos SQLite
def connect_db():
    try:
        conn = sqlite3.connect('../data/database/airline_satisfaction.db') # Conectar a la base de datos
        return conn
    except Exception as e:
        print(f"Error al conectar a la base de datos: {e}")
        return None

## Función para Crear y Insertar datos a la Tabla "modelos_entrenados" en SQLite
- Objetivo: Crear una tabla en SQLite para almacenar la información de los modelos.
- Razón: Facilitar el almacenamiento de las métricas y parámetros clave de cada modelo entrenado para su posterior consulta.

In [22]:
# Función para crear la tabla de modelos entrenados si no existe en la base de datos
def create_table():
    conn = connect_db()
    if conn:
        try:
            cursor = conn.cursor()
            cursor.execute('''
            CREATE TABLE IF NOT EXISTS modelos_entrenados (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                modelo TEXT,
                accuracy_mean REAL,
                precision_mean REAL,
                recall_mean REAL,
                f1_mean REAL,
                roc_auc_mean REAL,
                tiempo_procesamiento TEXT,
                archivo_modelo TEXT,
                n_estimators_catboost INTEGER,
                depth_catboost INTEGER,
                learning_rate_catboost REAL,
                n_estimators_rf INTEGER,
                n_estimators_xgb INTEGER,
                depth_xgb INTEGER,
                learning_rate_xgb REAL,
                fecha_entrenamiento TEXT
            )
            ''')
            conn.commit()
            conn.close()
            print("Tabla 'modelos_entrenados' creada o ya existente.")
        except Exception as e:
            print(f"Error al crear la tabla: {e}")

# Función para insertar un registro con el resultado del modelo en la base de datos
def insertar_modelo(modelo_nombre, accuracy, precision, recall, f1, roc_auc, tiempo, archivo, 
                    n_estimators_catboost=None, depth_catboost=None, learning_rate_catboost=None, 
                    n_estimators_rf=None, n_estimators_xgb=None, depth_xgb=None, learning_rate_xgb=None):
    conn = connect_db()
    if conn:
        try:
            cursor = conn.cursor()
            fecha_actual = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            cursor.execute('''
                INSERT INTO modelos_entrenados 
                (modelo, accuracy_mean, precision_mean, recall_mean, f1_mean, roc_auc_mean, 
                tiempo_procesamiento, archivo_modelo, n_estimators_catboost, depth_catboost, 
                learning_rate_catboost, n_estimators_rf, n_estimators_xgb, depth_xgb, 
                learning_rate_xgb, fecha_entrenamiento)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (modelo_nombre, accuracy, precision, recall, f1, roc_auc, tiempo, archivo, 
                  n_estimators_catboost, depth_catboost, learning_rate_catboost, n_estimators_rf, 
                  n_estimators_xgb, depth_xgb, learning_rate_xgb, fecha_actual))
            conn.commit()
            conn.close()
        except Exception as e:
            print(f"Error al insertar datos: {e}")

# Crear la tabla (si no existe) antes de comenzar la evaluación de modelos
create_table()


Tabla 'modelos_entrenados' creada o ya existente.


## Carga de Datos y Preprocesamiento
- Objetivo: Cargar y limpiar el dataset, eliminando valores nulos y columnas innecesarias.
- Razón: Un preprocesamiento adecuado es esencial para que los modelos funcionen correctamente y no haya errores de inconsistencia en los datos.

In [23]:
# Paso 1: Cargar los datos (CSV original)
data = pd.read_csv('../data/airline_passenger_satisfaction.csv')

# Paso 1.1: Eliminar registros con valores nulos en la columna 'Arrival Delay in Minutes'
data = data.dropna(subset=['Arrival Delay in Minutes'])

# Paso 1.2: Eliminar las columnas 'Unnamed: 0' y 'id' porque no aportan valor al análisis y Machine Learning
data = data.drop(['Unnamed: 0', 'id'], axis=1)

# Paso 1.3: Convertir las etiquetas de 'satisfaction' de cadenas a valores numéricos (0 y 1)
data['satisfaction'] = data['satisfaction'].map({'satisfied': 1, 'neutral or dissatisfied': 0})
y = data['satisfaction']
X = data.drop('satisfaction', axis=1)

## Codificación de Variables Categóricas
- Objetivo: Convertir las variables categóricas en variables numéricas utilizando un mapeo manual.
- Razón: Los algoritmos de machine learning generalmente no pueden trabajar directamente con datos categóricos, por lo que deben ser codificados numéricamente.

---
## <font color=#FF5733>¡Importante!</font>
- Para las variables (columnas) numericas ordinales no ha sido necesario la tranformación categorica porque los valores ya son numéricos en estas variables y ya poseen una orden de 0 a 5.

- Para evitar errores al comparar CatBoost con XGBoost y Random Forest, tuvimos que transformar las columnas categóricas usando una técnica llamada "map", en lugar de usar OneHotEncoder.

    El problema surgía porque CatBoost maneja automáticamente las variables categóricas, mientras que XGBoost y Random Forest no lo hacen. Para estos dos > algoritmos, necesitábamos transformar las variables categóricas usando OneHotEncoder, lo que creaba columnas adicionales en el conjunto de datos. Esto causaba errores al comparar los modelos porque CatBoost no esperaba esas columnas adicionales en su proceso.

    Para solucionar el problema y mantener la consistencia entre los modelos, decidimos usar el método "map" para codificar las variables categóricas en todos los algoritmos sin generar nuevas columnas. Esto permitió que el tamaño del conjunto de datos se mantuviera igual para los tres algoritmos y evitó los errores de comparación.

    En resumen, usamos "map" para transformar las variables categóricas de forma que los datos se mantuvieran compatibles entre todos los algoritmos sin cambiar la estructura del dataset.
---

In [24]:
# Paso 2: Codificar manualmente las variables categóricas usando `map`
categorical_cols = ['Gender', 'Customer Type', 'Type of Travel', 'Class']

# Diccionarios de mapeo para convertir valores categóricos a numéricos
categorical_mappings = {
    'Gender': {'Male': 0, 'Female': 1},
    'Customer Type': {'Loyal Customer': 1, 'disloyal Customer': 0},
    'Type of Travel': {'Business travel': 1, 'Personal Travel': 0},
    'Class': {'Eco': 0, 'Eco Plus': 1, 'Business': 2}
}

# Aplicar las transformaciones a las columnas categóricas
for col, mapping in categorical_mappings.items():
    X[col] = X[col].map(mapping)

# Definir las columnas numéricas y ordinales
ordinal_cols = ['Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location', 
                'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 
                'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness']


# Preprocesamiento: Escalar las columnas numéricas para los algoritmos que necesitan del escalado (CatBoost no necesita)
numerical_cols = ['Age', 'Flight Distance', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols)
    ]
)


## Configuración de Pipelines para Modelos individuales (RandomForestClassifier, XGBoost, y CatBoost)
- Objetivo: Crear pipelines de preprocesamiento y modelado para facilitar la aplicación de transformaciones y entrenamiento de los modelos.
- Razón: Los pipelines permiten una mayor modularidad y limpieza en la ejecución de tareas repetitivas, como el escalado y ajuste de modelos.

Concepto Pipeline:
- Un pipeline te permite encadenar varios pasos de preprocesamiento y el modelo de machine learning en una secuencia, asegurando que todos los pasos se ejecuten de manera consistente. Esto es útil cuando tienes que aplicar varias transformaciones (como la codificación de variables categóricas, imputación de valores nulos, etc.) antes de entrenar el modelo.
- En este cuaderno luego abajo aplicamos el "Escalado de las variables numéricas" para los algoritmos que necesitan del escalado (CatBoost no necesita)

In [25]:
# Paso 4: Crear los pipelines específicos de cada modelo
pipeline_rf = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', RandomForestClassifier(n_estimators=100, random_state=42))
])

pipeline_xgb = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss'))
])

pipeline_catboost = Pipeline(steps=[
    ('model', CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, verbose=False))
])


## Técnica de Ensemble Learning - StackingClassifier
- Este es un paso importante porque estamos combinando múltiples modelos para mejorar el rendimiento.
- Explicación: El StackingClassifier combina las predicciones de varios modelos base (CatBoost, XGBoost, RandomForest) y las pasa a un modelo meta (otro RandomForest en este caso). El objetivo es mejorar la precisión y la capacidad de generalización del modelo.

¿Qué es el modelo de Stacking?
El modelo de Stacking es un ensemble que combina las predicciones de varios modelos base (en este caso, Random Forest, XGBoost, y CatBoost) mediante un modelo final, que es conocido como meta-modelo. Este meta-modelo toma las predicciones de los modelos base y produce una predicción final más precisa.

En este caso, el modelo que estamos guardando es el modelo completo de Stacking que utiliza estos tres modelos base y un Random Forest como meta-modelo para hacer la predicción final.

¿Qué ocurre con los modelos individuales?
El código actual evalúa el rendimiento de cada modelo individual (Random Forest, XGBoost, y CatBoost) y también evalúa el modelo de Stacking. Los resultados se almacenan en un DataFrame y se imprimen, pero no estamos seleccionando automáticamente el modelo individual con mejor rendimiento para guardarlo por separado. Actualmente, solo se guarda el modelo de Stacking completo, independientemente de si este tiene el mejor rendimiento en comparación con los modelos individuales.


In [26]:
# Modelo de Stacking que combina los tres modelos anteriores
stacking_model = StackingClassifier(
    estimators=[
        ('catboost', pipeline_catboost),
        ('xgboost', pipeline_xgb),
        ('random_forest', pipeline_rf)
    ],
    final_estimator=RandomForestClassifier(n_estimators=50, random_state=42), # Meta-modelo: Random Forest
    cv=5 # Validación cruzada 5 pligues
)

## Evaluación y Validación Cruzada
- Objetivo: Evaluar cada modelo utilizando validación cruzada estratificada y calcular el tiempo de procesamiento.
- Razón: La validación cruzada permite obtener una estimación más robusta del rendimiento del modelo, y medir el tiempo ayuda a tener en cuenta la eficiencia de cada algoritmo.

- Concepto Validación Cruzada:
La validación cruzada es una técnica usada en machine learning para evaluar el rendimiento de un modelo de manera más robusta. En lugar de entrenar y probar el modelo una sola vez dividiendo los datos en un conjunto de entrenamiento y otro de prueba, la validación cruzada divide los datos en varios subconjuntos y realiza múltiples ciclos de entrenamiento y prueba, lo que permite obtener una evaluación más estable y evitar sobreajuste (overfitting).

In [27]:
# Definir las métricas de evaluación
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'roc_auc': make_scorer(roc_auc_score)
}

# Función para evaluar el modelo con tiempo de procesamiento
def evaluar_modelo_con_tiempo(pipeline, X, y, cv):
    start_time = time.time()
    cv_results = cross_validate(pipeline, X, y, cv=cv, scoring=scoring, return_train_score=False)
    end_time = time.time()
    elapsed_time = end_time - start_time
    elapsed_time_readable = str(timedelta(seconds=elapsed_time))
    
    return cv_results, elapsed_time, elapsed_time_readable

# Definir el KFold estratificado para la validación cruzada
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Lista para almacenar los resultados de los modelos
resultados = []

# Diccionario con los modelos que vamos a evaluar
modelos = {
    'Random Forest': pipeline_rf,
    'XGBoost': pipeline_xgb,
    'CatBoost': pipeline_catboost,
    'Stacking': stacking_model
}

# Evaluar cada modelo con validación cruzada y almacenar los resultados
for nombre_modelo, modelo in modelos.items():
    print(f"\nEvaluando {nombre_modelo}...")
    cv_resultados, tiempo, tiempo_legible = evaluar_modelo_con_tiempo(modelo, X, y, kf)
    
    # Almacena los resultados en el diccionario
    resultados.append({
        'modelo': nombre_modelo,
        'accuracy_mean': cv_resultados['test_accuracy'].mean(),
        'precision_mean': cv_resultados['test_precision'].mean(),
        'recall_mean': cv_resultados['test_recall'].mean(),
        'f1_mean': cv_resultados['test_f1'].mean(),
        'roc_auc_mean': cv_resultados['test_roc_auc'].mean(),
        'tiempo_procesamiento': tiempo,
        'tiempo_procesamiento_legible': tiempo_legible,
        'modelo_pipeline': modelo
    })

# Crear un DataFrame con los resultados y mostrarlo
df_resultados = pd.DataFrame(resultados)



Evaluando Random Forest...

Evaluando XGBoost...

Evaluando CatBoost...

Evaluando Stacking...


## Selección del Mejor Modelo y Almacenamiento
- Objetivo: Seleccionar el modelo con el mejor rendimiento en función de la métrica de accuracy.
- Razón: Este paso es clave para escoger el modelo que mejor se adapte a los datos, basándose en las métricas de evaluación seleccionadas.

In [28]:
# Eliminar la columna 'modelo_pipeline' para una mejor visualización en consola
df_vista_consola = df_resultados.drop(columns=['modelo_pipeline'])

# Mostrar los resultados finales en la consola
print("\nResultados finales:")
print(tabulate(df_vista_consola, headers='keys', tablefmt='psql'))

# Guardar los resultados en un archivo CSV
output_csv = '../data/modelos_entrenamiento/resultados_modelos_completos.csv'
df_resultados.to_csv(output_csv, index=False)
print(f"\nLos resultados completos se han guardado en el archivo: {output_csv}")

# Identificar el mejor modelo basado en el accuracy
mejor_modelo = df_resultados.loc[df_resultados['accuracy_mean'].idxmax()] # Obtener el mejor modelo basado en la precisión

print(f"\nEl mejor modelo ha sido: {mejor_modelo['modelo']} debido a su mejor rendimiento en accuracy de {mejor_modelo['accuracy_mean']:.4f}")
print(f"\nEntrenando el mejor modelo: {mejor_modelo['modelo']} con todos los datos...")



Resultados finales:
+----+---------------+-----------------+------------------+---------------+-----------+----------------+------------------------+--------------------------------+
|    | modelo        |   accuracy_mean |   precision_mean |   recall_mean |   f1_mean |   roc_auc_mean |   tiempo_procesamiento | tiempo_procesamiento_legible   |
|----+---------------+-----------------+------------------+---------------+-----------+----------------+------------------------+--------------------------------|
|  0 | Random Forest |        0.632373 |         0.583962 |      0.527764 |  0.55444  |       0.620075 |              136.41    | 0:02:16.410275                 |
|  1 | XGBoost       |        0.677925 |         0.674597 |      0.496225 |  0.571813 |       0.656566 |                4.33325 | 0:00:04.333248                 |
|  2 | CatBoost      |        0.960191 |         0.968753 |      0.938415 |  0.953341 |       0.957631 |               17.6212  | 0:00:17.621177                 |
|

## Entrenamiento Final, Almacenamiento del Mejor Modelo y Registro de sus Parámetros
- Objetivo: Entrenar el mejor modelo y guardarlo en disco y registrar sus parámetros en la base de datos SQLite.
- Razón: Guardar el modelo permite reutilizarlo en el futuro sin necesidad de volver a entrenarlo, mientras que almacenar sus métricas y parámetros facilita el seguimiento del rendimiento de los modelos.


In [29]:
# Finalmente, entrenamos el modelo con todos los datos y lo almacenamos en el sistema de archivos. También guardamos sus parámetros y métricas en la base de datos.
modelo_final = mejor_modelo['modelo_pipeline'] # Extraer el pipeline del mejor modelo
modelo_final.fit(X, y) # Entrenar el modelo final con todos los datos

accuracy = mejor_modelo['accuracy_mean']
precision = mejor_modelo['precision_mean']
recall = mejor_modelo['recall_mean']
f1 = mejor_modelo['f1_mean']
roc_auc = mejor_modelo['roc_auc_mean']
tiempo_procesamiento = mejor_modelo['tiempo_procesamiento_legible']
archivo_modelo = "" # Inicializar el nombre del archivo para guardar el modelo

# Función para guardar el modelo con un nombre apropiado
def guardar_modelo_con_nombre(modelo, nombre_modelo):
    output_dir = '../data/modelos_entrenamiento/' # Directorio donde se guardarán los modelos
    os.makedirs(output_dir, exist_ok=True) # Crear el directorio si no existe
    
    fecha_hora_actual = datetime.now().strftime('%Y%m%d_%H%M%S') # Obtener la fecha y hora actual
    
    if nombre_modelo == 'CatBoost':
        archivo_modelo = f"{nombre_modelo}_mejor_modelo_{fecha_hora_actual}.cbm"  # Guardar en formato específico de CatBoost
        modelo.named_steps['model'].save_model(os.path.join(output_dir, archivo_modelo)) # Guardar el modelo de CatBoost
        print(f"Modelo CatBoost guardado exitosamente con el nombre: {archivo_modelo}")
    else:
        archivo_modelo = f"{nombre_modelo}_mejor_modelo_{fecha_hora_actual}.pkl" # Guardar en formato pickle
        joblib.dump(modelo, os.path.join(output_dir, archivo_modelo)) # Guardar el modelo con joblib    
        print(f"Modelo guardado exitosamente con el nombre: {archivo_modelo}")
    
    return archivo_modelo # Retornar el nombre del archivo guardado

# Guardar el mejor modelo y obtener el nombre del archivo
archivo_modelo = guardar_modelo_con_nombre(modelo_final, mejor_modelo['modelo'])

# Inicializar parámetros específicos para los diferentes modelos
n_estimators_catboost = None
depth_catboost = None
learning_rate_catboost = None
n_estimators_rf = None
n_estimators_xgb = None
depth_xgb = None
learning_rate_xgb = None

# Obtener los parámetros relevantes del mejor modelo
if mejor_modelo['modelo'] == 'CatBoost':
    n_estimators_catboost = modelo_final.named_steps['model'].get_params().get('iterations')
    depth_catboost = modelo_final.named_steps['model'].get_params().get('depth')
    learning_rate_catboost = modelo_final.named_steps['model'].get_params().get('learning_rate')
elif mejor_modelo['modelo'] == 'Random Forest':
    n_estimators_rf = modelo_final.named_steps['model'].get_params().get('n_estimators')
elif mejor_modelo['modelo'] == 'XGBoost':
    n_estimators_xgb = modelo_final.named_steps['model'].get_params().get('n_estimators')
    depth_xgb = modelo_final.named_steps['model'].get_params().get('max_depth')
    learning_rate_xgb = modelo_final.named_steps['model'].get_params().get('learning_rate')

# Insertar los resultados y parámetros del mejor modelo en la base de datos
insertar_modelo(mejor_modelo['modelo'], accuracy, precision, recall, f1, roc_auc, tiempo_procesamiento, archivo_modelo, 
                n_estimators_catboost, depth_catboost, learning_rate_catboost, 
                n_estimators_rf, n_estimators_xgb, depth_xgb, learning_rate_xgb)

# Mensaje final indicando que el modelo y sus parámetros han sido almacenados en la base de datos
print(f"Modelo {mejor_modelo['modelo']} y sus parámetros han sido almacenados en la base de datos.")


Modelo CatBoost guardado exitosamente con el nombre: CatBoost_mejor_modelo_20240910_123529.cbm
Modelo CatBoost y sus parámetros han sido almacenados en la base de datos.
